[paper with code](https://paperswithcode.com/sota/image-classification-on-mnist?metric=Accuracy) 并没有纯CNN 方法，
不如考虑对比LeNet,AlexNet,VGG,ResNet等方法。 
Ref to [https://zh.d2l.ai/chapter_convolutional-modern/index.html](https://zh.d2l.ai/chapter_convolutional-modern/index.html)

目前主流的对比数据集是 3 个：[Binary Connect](https://arxiv.org/pdf/1511.00363) 和 [	
ReNet](https://arxiv.org/pdf/1505.00393v3)

# MedMnist
[]()

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from os.path import join
from config.config import load_config28

In [ ]:
dataset_name = "organmnist3d"
config = load_config28(dataset_name)

In [ ]:
ORDER = config["order"]
SIZE = config["data"]["size"]
TASK = config["task"]
NUM_CLASSES = config["network"]["num_classes"]
EPOCHS = config["train"]["epochs"]
BATCH_SIZE = config["data"]["batch_size"]
INPUT_CHANNEL = config["network"]["input_channels"]
LINEAR_SIZE = config["network"]["linear_size"]
CRITERION = config["train"]["criterion"]
DEVICE = config["train"]["device"]
experiment_name = f"{ORDER}-{dataset_name}-{SIZE}"

In [ ]:
checkpoint_save_dir = join(config["train"]["checkpoint_save_dir"], experiment_name)

In [ ]:
from data_load import get_dataloader

train_loader, test_loader = get_dataloader(dataset_name, BATCH_SIZE, SIZE, TASK)

In [ ]:
from tcnn.utils.experiment.model import (
    get_optimizers_dict_medmnist,
    get_schedulers_dict_medmnist,
)
import lenet_models
import lenet_models_binary

if TASK == "multiclass":
    model_dict = lenet_models.get_model_dict(INPUT_CHANNEL, NUM_CLASSES, LINEAR_SIZE)
    print("Loaded multiclass model with {} classes".format(NUM_CLASSES))
elif TASK == "binary":
    model_dict = lenet_models_binary.get_model_dict(INPUT_CHANNEL, LINEAR_SIZE)
    print("Loaded binary model")
optimizers_dict = get_optimizers_dict_medmnist(model_dict)
schedulers_dict = get_schedulers_dict_medmnist(optimizers_dict)

In [ ]:
history_dict = {}

In [ ]:
from tcnn.utils.experiment.train import train_and_test_model

for model_name, model in model_dict.items():
    print(f"Training model {model_name}")
    history_dict[model_name] = train_and_test_model(
        model,
        train_loader,
        test_loader,
        CRITERION,
        optimizers_dict[model_name],
        scheduler=schedulers_dict[model_name],
        epochs=EPOCHS,
        save_checkpoint=True,
        save_checkpoint_interval=1,
        checkpoint_save_dir=join(checkpoint_save_dir, model_name),
        task=TASK,
    )
    print("***" * 10)

In [ ]:
from tcnn.utils.experiment.plot import plot_history

for model_name, history in history_dict.items():
    print(f"Model {model_name} history:")
    plot_history(history, model_name)

In [ ]:
import torch

input_shape = (BATCH_SIZE, 1, SIZE, SIZE, SIZE)
input_tensor = torch.randn(input_shape).to(DEVICE)

In [ ]:
import torchprofile
from tcnn.utils.experiment.model import count_parameters
from tcnn.utils.experiment.train import eval_model

result = dict()
for model_name, model in model_dict.items():
    print(f"Evaluating model {model_name}")
    result[model_name] = dict()
    result[model_name]["macs"] = torchprofile.profile_macs(model, input_tensor)
    result[model_name]["params"] = count_parameters(model)
    result[model_name]["performance"] = eval_model(model, test_loader, CRITERION, TASK)
    with torch.no_grad():
        torch.cuda.empty_cache()
    print("***" * 10)

In [ ]:
from tcnn.utils.experiment.plot import plot_history_dict

plot_history_dict(history_dict)

In [ ]:
from tcnn.utils.experiment.log import show_test_result

show_test_result(result)

In [ ]:
with torch.no_grad():
    torch.cuda.empty_cache()